In [27]:
import pymysql
import pandas as pd
from IPython.display import display, clear_output
from ipywidgets import Dropdown
connect = pymysql.connect(host='127.0.0.1', user='root', password='0000', db='practice', charset='utf8')
cur = connect.cursor()

test = "SELECT * FROM practice.m_area;"
test1 = "SELECT * FROM practice.m_store;"
query1 = "SELECT * FROM practice.tbl_order_202105;"

cur.execute(test)
test = pd.DataFrame(cur.fetchall())
test.columns = ['area_cd', 'wide_area', 'narrow_area']

cur.execute(test1)
test1 = pd.DataFrame(cur.fetchall())
test1.columns = ['store_id', 'store_name', 'area_cd']


cur.execute(query1)
data1 = pd.DataFrame(cur.fetchall())
data1.columns = ['order_id', 'store_id', 'customer_id', 'coupon_cd', 'sales_detail_id',
                 'order_accept_date', 'delivered_date', 'takeout_flag', 'total_amount', 'status']


In [43]:
import glob
import os

current_dir = os.getcwd()
tbl_order_file = os.path.join(current_dir,'tbl_order_*.csv')
tbl_order_files = glob.glob(tbl_order_file)

order_all = pd.DataFrame()
for file in tbl_order_files:
    order_tmp = pd.read_csv(file)
    print(f'{file}:{len(order_tmp)}')
    order_all = pd.concat([data1,order_tmp],ignore_index=True)

order_all = data1.loc[data1['store_id']!=999]

order_all = pd.merge(data1,test1,on='store_id',how='left')
order_all = pd.merge(order_all,test,on='area_cd',how='left')

order_all.loc[order_all['takeout_flag']==0,'takeout_name'] = 'delivery'
order_all.loc[order_all['takeout_flag']==1,'takeout_name'] = 'takeout'

order_all.loc[order_all['status']==0,'status_name'] = '주문 접수'
order_all.loc[order_all['status']==0,'status_name'] = '결제 완료'
order_all.loc[order_all['status']==0,'status_name'] = '배달 완료'
order_all.loc[order_all['status']==0,'status_name'] = '주문 취소'

order_all.loc[:,'order_date'] = pd.to_datetime(
    order_all['order_accept_date']).dt.date

order_all.groupby(['store_id','customer_id'])['total_amount'].describe()

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name,order_date
0,84536666,16,C09876770,89,98297032,2021-05-01 11:00,2021-05-01 11:23,0,32340,2,삼봉로점,SL,서울,서울,delivery,NaN,2021-05-01
1,88276008,158,C68585017,98,7770950,2021-05-01 11:00,2021-05-01 11:27,0,38650,2,광산로점,GJ,광주,광주,delivery,NaN,2021-05-01
2,30853149,97,C47161470,20,70091383,2021-05-01 11:00,2021-05-01 11:51,0,28270,2,연제로점,BS,부산,부산,delivery,NaN,2021-05-01
3,2266595,95,C61461123,22,42134845,2021-05-01 11:00,2021-05-01 11:39,0,23160,2,중앙대로2점,BS,부산,부산,delivery,NaN,2021-05-01
4,7086301,49,C01007474,99,69495399,2021-05-01 11:00,2021-05-01 11:37,0,19000,9,당산로2점,SL,서울,서울,delivery,NaN,2021-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241137,80834230,110,C44748299,25,61000508,2021-05-31 21:58,2021-05-31 22:42,0,35020,2,중앙대로점,BS,부산,부산,delivery,NaN,2021-05-31
241138,65940233,164,C68733497,33,29474794,2021-05-31 21:58,2021-05-31 22:08,0,21120,2,용아로점,GJ,광주,광주,delivery,NaN,2021-05-31
241139,49922027,23,C66722860,92,96482427,2021-05-31 21:58,2021-05-31 22:27,0,26470,2,시흥대로점,SL,서울,서울,delivery,NaN,2021-05-31
241140,80642623,35,C32096353,4,88688082,2021-05-31 21:58,2021-05-31 22:40,0,28270,2,국제금융로점,SL,서울,서울,delivery,NaN,2021-05-31
